In [70]:
#auto reload modules
%load_ext autoreload
%autoreload 2

from datetime import datetime
import os
import random
import json
from src.llm.PredictionUtils import init_model_and_tokenizer,llm_gen
from src.prop.utils import gen_problem,parse_prediction,gen_reason,gen_masked_prediction_problem_prompts,parse_prediction_with_check

pid = os.getpid()
seed = int(pid)+int(datetime.now().timestamp())
print("seed: ", seed)
random.seed(seed)



out_dir = "data"
os.system(f"mkdir -p {out_dir}")

current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
seed:  1727232278


In [2]:
model_id="meta-llama/Meta-Llama-3.1-70B-Instruct"
model,tokenizer=init_model_and_tokenizer(model_id)

INFO 09-18 15:46:02 config.py:904] Defaulting to use mp for distributed inference
INFO 09-18 15:46:02 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-70B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-70B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=8, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llam

Loading safetensors checkpoint shards:   0% Completed | 0/30 [00:00<?, ?it/s]


INFO 09-18 15:46:21 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582848) INFO 09-18 15:46:22 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582851) INFO 09-18 15:46:22 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582846) INFO 09-18 15:46:22 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582847) INFO 09-18 15:46:23 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582850) INFO 09-18 15:46:23 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582845) INFO 09-18 15:46:24 model_runner.py:1008] Loading model weights took 16.4605 GB
(VllmWorkerProcess pid=582849) INFO 09-18 15:46:24 model_runner.py:1008] Loading model weights took 16.4605 GB
INFO 09-18 15:46:28 distributed_gpu_executor.py:57] # GPU blocks: 82648, # CPU blocks: 6553
(VllmWorkerProcess pid=582850) INFO 09-18 15:46:30 

In [81]:
from datasets import load_dataset

ds=load_dataset("kanhatakeyama/material-properties",split="train")
ds=ds.shuffle(seed=seed)
len(ds)

7241

In [82]:
train_ds=ds.select(range(7000))
test_ds=ds.select(range(7000,7200))
train_ds[0]

{'CompName': 'unknown',
 'SMILES': '[O-]S(=O)(=O)[O-].[Cs+].[Cs+]',
 'Property': 'Absolute standard enthalpy of formation',
 'Value': 1442.9,
 'unit': '[kJ/mol]',
 'Source': 'Wikipedia/Wikidata'}

In [85]:
from tqdm import tqdm
n_records=100

generated_records=[]
for _ in tqdm(range(30)):
    problems=[]
    prompt_list=[
    ]
    for i in range(n_records):
        record=random.choice(train_ds)
        #prompt,actual_value=gen_problem(record,tokenizer)
        prompt,actual_value=gen_reason(record,tokenizer)
        problems.append(
            {
            "record":record,
            "prompt":prompt,
            "actual_value":actual_value   
            }

        )
        prompt_list.append(prompt)

    #interference
    predicted_text_list=llm_gen(model,prompt_list)
    
    #predict properties according to the generated reasonings.
    check_prompt_list,masked_reason_list=gen_masked_prediction_problem_prompts(predicted_text_list,problems,tokenizer)
    predicted_value_list=llm_gen(model,check_prompt_list)


    #parse result
    #prediction_records=parse_prediction(problems,predicted_text_list)
    prediction_records=parse_prediction_with_check(problems, masked_reason_list, predicted_value_list)

    generated_records.extend(prediction_records)
    for i in range(len(prediction_records)):
        with open(f"data/{current_time_no_symbols}_llm_gen.jsonl","a", encoding='utf-8') as f:
            f.write(json.dumps(prediction_records[i],ensure_ascii=False)+"\n")

  0%|          | 0/30 [00:00<?, ?it/s]

 40%|████      | 12/30 [04:49<07:51, 26.20s/it]

In [78]:
threshold=0.1

good_predictions=[r for r in prediction_records if r["error_rate"]<threshold]
len(good_predictions),good_predictions

(43,
 [{'record': {'CompName': 'Chromium(III) oxide',
    'SMILES': 'O=[Cr]O[Cr]=O',
    'Property': 'Absolute standard enthalpy of formation',
    'Value': 1128.0,
    'unit': '[kJ/mol]',
    'Source': 'Wikipedia/Wikidata'},
   'prompt': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nProvide the quantitative Reason and Prediction so that a scientist,\nwho does not know the Absolute standard enthalpy of formation [kJ/mol] of Chromium(III) oxide (O=[Cr]O[Cr]=O), can predict the value.\n\n#Commands\nfocusing on each functional groups.\n- Actual value and Prediction must match each other.\n- Your answer must contain #Reason and #Prediction section.\n- Never include the actual value (1128.0 [kJ/mol]) in #Reason, but include it in #Prediction.\n- #Prediction must contain only the predicted value (number only).<|eot_id|>assistant\n\n#Reason\n',
   'actual_v

In [79]:
import pandas as pd
df=pd.DataFrame(good_predictions)
df=df.drop(columns=["record","prompt"])
#df.to_csv("predicted.csv",index=False)

In [80]:
df

,actual_value,CompoundName,SMILES,Property,Unit,predicted,reason,error_rate
0,1128.000000,Chromium(III) oxide,O=[Cr]O[Cr]=O,Absolute standard enthalpy of formation,[kJ/mol],1128,To predict the standard enthalpy of formation ...,0.000000
1,0.801000,Borane dimethylsulfide,[BH3-][S+](C)C,Density,[g/cm3],0.725,"To predict the density of Compound XC), we can...",0.094881
2,1.497000,Etridiazole,CCOC1=NC(=NS1)C(Cl)(Cl)Cl,Density,[g/cm3],1.50,"To predict the density of Compound X, let's an...",0.002004
3,230.000000,Nickel(II) hydroxide,[Ni+2].[OH-].[OH-],Melting temperature,[oC],230,"Compound X, Ni(OH)2, is an ionic compound cons...",0.000000
4,525.000000,Mercury(I) chloride,Cl[Hg][Hg]Cl,Melting temperature,[oC],526,Compound X is an ionic compound consisting of ...,0.001905
5,1.056000,Ethyl carbamate,CCOC(N)=O,Density,[g/cm3],1.05,"To predict the density of Compound X, we need ...",0.005682
6,10.590000,unknown,CC1=C(C=C(C=C1[N+](=O)[O-])[N+](=O)[O-])[N+](=...,Ionization Energy,[eV],11.5,To predict the ionization energy of the given ...,0.085930
7,1.498000,unknown,CC(=O)OCCC1=CC=CC=C1,Refractive index,[-],1.48,The refractive index of a compound can be pred...,0.012016
8,0.801000,Borane dimethylsulfide,[BH3-][S+](C)C,Density,[g/cm3],0.78,"To predict the density of Compound XC), we can...",0.026217
9,1.416400,unknown,CC(Cl)Cl,Refractive index,[-],1.50,The refractive index of a compound can be pred...,0.059023
